In [ ]:
!pip install pytorch_tabnet

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('seaborn')
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense,LeakyReLU,BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from keras.layers import Dropout,LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.utils import SparsePredictDataset, PredictDataset, filter_weights
from pytorch_tabnet.abstract_model import TabModel
from pytorch_tabnet.multiclass_utils import infer_output_dim, check_output_dim
from torch import optim
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
d = pd.read_csv('/content/drive/MyDrive/creditcard.csv')

In [ ]:
data = d.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data[['Amount']])
X = data.drop(['Class'],axis=1)
Y = data.Class

### 1

In [ ]:
pretrainer1 = TabNetPretrainer(
    n_d=48, n_a=48,  # 피처 차원 수 증가
    optimizer_fn=torch.optim.AdamW,  # AdamW로 변경
    optimizer_params=dict(lr=5e-3, weight_decay=1e-3),  # 학습률 감소 및 정규화 추가
    mask_type="entmax",  # sparsemax 대신 entmax 사용
    lambda_sparse=1e-4  # 정규화 감소
)

pretrainer1.fit(
    X_train=X.values,
    eval_set=[X.values],  # 평가 데이터 (자기지도 학습이므로 동일 데이터 사용)
    max_epochs=200,
    patience=300,
    batch_size=512
)

epoch 0  | loss: 1.63306 | val_0_unsup_loss_numpy: 0.8825700283050537|  0:00:37s
epoch 1  | loss: 0.85722 | val_0_unsup_loss_numpy: 0.782480001449585|  0:01:14s
epoch 2  | loss: 0.78481 | val_0_unsup_loss_numpy: 0.7570899724960327|  0:01:50s
epoch 3  | loss: 0.72218 | val_0_unsup_loss_numpy: 0.6301699876785278|  0:02:27s
epoch 4  | loss: 0.67478 | val_0_unsup_loss_numpy: 0.6181899905204773|  0:03:03s
epoch 5  | loss: 0.64829 | val_0_unsup_loss_numpy: 0.6113799810409546|  0:03:39s
epoch 6  | loss: 0.62828 | val_0_unsup_loss_numpy: 0.6534000039100647|  0:04:15s
epoch 7  | loss: 0.61098 | val_0_unsup_loss_numpy: 0.5326799750328064|  0:04:51s
epoch 8  | loss: 0.59534 | val_0_unsup_loss_numpy: 0.5438699722290039|  0:05:26s
epoch 9  | loss: 0.57931 | val_0_unsup_loss_numpy: 0.4743199944496155|  0:06:04s
epoch 10 | loss: 0.5687  | val_0_unsup_loss_numpy: 0.53329998254776|  0:06:40s
epoch 11 | loss: 0.56135 | val_0_unsup_loss_numpy: 0.6880499720573425|  0:07:16s
epoch 12 | loss: 0.55422 | val_

KeyboardInterrupt: 

In [ ]:
reconstructed_data = pretrainer1.predict(X.values)[0]
mse_X = np.mean(np.power(X - reconstructed_data, 2), axis=1)
mae_X = np.mean(np.abs(X - reconstructed_data), axis=1)

data['mse_X'] = mse_X
data['mae_X'] = mae_X

n = 1000
df = data.sort_values(by='mae_X', ascending=False)
top_n_df = df.head(n)
accuracy = top_n_df[top_n_df['Class'] == 1].shape[0] / n
print('Top-{} 정확도: {}'.format(n, accuracy))

Top-1000 정확도: 0.129


In [ ]:
pretrainer1.fit(
    X_train=X.values,
    eval_set=[X.values],  # 평가 데이터 (자기지도 학습이므로 동일 데이터 사용)
    max_epochs=200,
    patience=300,
    batch_size=512,
    warm_start=True
)

epoch 0  | loss: 0.43713 | val_0_unsup_loss_numpy: 0.46029001474380493|  0:00:39s
epoch 1  | loss: 0.43826 | val_0_unsup_loss_numpy: 0.44235000014305115|  0:01:14s
epoch 2  | loss: 0.43674 | val_0_unsup_loss_numpy: 0.41093000769615173|  0:01:52s
epoch 3  | loss: 0.43462 | val_0_unsup_loss_numpy: 0.3617599904537201|  0:02:28s
epoch 4  | loss: 0.4299  | val_0_unsup_loss_numpy: 0.4868899881839752|  0:03:05s
epoch 5  | loss: 0.43007 | val_0_unsup_loss_numpy: 0.5677300095558167|  0:03:40s
epoch 6  | loss: 0.4295  | val_0_unsup_loss_numpy: 0.43748000264167786|  0:04:16s
epoch 7  | loss: 0.43047 | val_0_unsup_loss_numpy: 0.3778499960899353|  0:04:53s
epoch 8  | loss: 0.42674 | val_0_unsup_loss_numpy: 0.3378300070762634|  0:05:30s
epoch 9  | loss: 0.42388 | val_0_unsup_loss_numpy: 0.33678001165390015|  0:06:05s
epoch 10 | loss: 0.42564 | val_0_unsup_loss_numpy: 0.3833000063896179|  0:06:41s
epoch 11 | loss: 0.42483 | val_0_unsup_loss_numpy: 0.44637998938560486|  0:07:17s
epoch 12 | loss: 0.428

KeyboardInterrupt: 

In [ ]:
reconstructed_data = pretrainer1.predict(X.values)[0]
mse_X = np.mean(np.power(X - reconstructed_data, 2), axis=1)
mae_X = np.mean(np.abs(X - reconstructed_data), axis=1)

data['mse_X'] = mse_X
data['mae_X'] = mae_X

n = 1000
df = data.sort_values(by='mae_X', ascending=False)
top_n_df = df.head(n)
accuracy = top_n_df[top_n_df['Class'] == 1].shape[0] / n
print('Top-{} 정확도: {}'.format(n, accuracy))

Top-1000 정확도: 0.014


In [ ]:
pretrainer1.fit(
    X_train=X.values,
    eval_set=[X.values],  # 평가 데이터 (자기지도 학습이므로 동일 데이터 사용)
    max_epochs=200,
    patience=300,
    batch_size=512,
    warm_start=True
)

epoch 0  | loss: 0.409   | val_0_unsup_loss_numpy: 0.6554099917411804|  0:00:36s
epoch 1  | loss: 0.40736 | val_0_unsup_loss_numpy: 0.8059099912643433|  0:01:14s
epoch 2  | loss: 0.40583 | val_0_unsup_loss_numpy: 0.9253399968147278|  0:01:51s
epoch 3  | loss: 0.40647 | val_0_unsup_loss_numpy: 0.6261699795722961|  0:02:28s
epoch 4  | loss: 0.40863 | val_0_unsup_loss_numpy: 0.33873000741004944|  0:03:04s
epoch 5  | loss: 0.40566 | val_0_unsup_loss_numpy: 0.6198999881744385|  0:03:41s
epoch 6  | loss: 0.40586 | val_0_unsup_loss_numpy: 0.9668400287628174|  0:04:17s
epoch 7  | loss: 0.40943 | val_0_unsup_loss_numpy: 0.7118300199508667|  0:04:55s
epoch 8  | loss: 0.40431 | val_0_unsup_loss_numpy: 0.967739999294281|  0:05:32s
epoch 9  | loss: 0.40659 | val_0_unsup_loss_numpy: 6.192140102386475|  0:06:08s
epoch 10 | loss: 0.40504 | val_0_unsup_loss_numpy: 1.4412000179290771|  0:06:44s
epoch 11 | loss: 0.40352 | val_0_unsup_loss_numpy: 0.5501800179481506|  0:07:21s
epoch 12 | loss: 0.40153 | va

KeyboardInterrupt: 

In [ ]:
reconstructed_data = pretrainer1.predict(X.values)[0]
mse_X = np.mean(np.power(X - reconstructed_data, 2), axis=1)
mae_X = np.mean(np.abs(X - reconstructed_data), axis=1)

data['mse_X'] = mse_X
data['mae_X'] = mae_X

n = 1000
df = data.sort_values(by='mae_X', ascending=False)
top_n_df = df.head(n)
accuracy = top_n_df[top_n_df['Class'] == 1].shape[0] / n
print('Top-{} 정확도: {}'.format(n, accuracy))

Top-1000 정확도: 0.169


In [ ]:
pretrainer1.fit(
    X_train=X.values,
    eval_set=[X.values],  # 평가 데이터 (자기지도 학습이므로 동일 데이터 사용)
    max_epochs=200,
    patience=300,
    batch_size=512,
    warm_start=True
)

epoch 0  | loss: 0.38552 | val_0_unsup_loss_numpy: 1.0139299631118774|  0:00:36s
epoch 1  | loss: 0.3846  | val_0_unsup_loss_numpy: 0.48475998640060425|  0:01:13s
epoch 2  | loss: 0.38726 | val_0_unsup_loss_numpy: 0.5764600038528442|  0:01:49s
epoch 3  | loss: 0.38672 | val_0_unsup_loss_numpy: 0.7396299839019775|  0:02:25s
epoch 4  | loss: 0.38478 | val_0_unsup_loss_numpy: 1.6481000185012817|  0:03:01s
epoch 5  | loss: 0.38328 | val_0_unsup_loss_numpy: 0.6000300049781799|  0:03:38s
epoch 6  | loss: 0.38723 | val_0_unsup_loss_numpy: 0.5336400270462036|  0:04:13s
epoch 7  | loss: 0.38564 | val_0_unsup_loss_numpy: 0.9137700200080872|  0:04:51s
epoch 8  | loss: 0.38336 | val_0_unsup_loss_numpy: 0.4390999972820282|  0:05:28s
epoch 9  | loss: 0.3845  | val_0_unsup_loss_numpy: 0.8257200121879578|  0:06:04s
epoch 10 | loss: 0.38157 | val_0_unsup_loss_numpy: 0.5666199922561646|  0:06:40s
epoch 11 | loss: 0.38744 | val_0_unsup_loss_numpy: 1.3922699689865112|  0:07:16s
epoch 12 | loss: 0.38372 | 

KeyboardInterrupt: 

In [ ]:
reconstructed_data = pretrainer1.predict(X.values)[0]
mse_X = np.mean(np.power(X - reconstructed_data, 2), axis=1)
mae_X = np.mean(np.abs(X - reconstructed_data), axis=1)

data['mse_X'] = mse_X
data['mae_X'] = mae_X

n = 1000
df = data.sort_values(by='mae_X', ascending=False)
top_n_df = df.head(n)
accuracy = top_n_df[top_n_df['Class'] == 1].shape[0] / n
print('Top-{} 정확도: {}'.format(n, accuracy))

Top-1000 정확도: 0.274


In [ ]:
pretrainer1.fit(
    X_train=X.values,
    eval_set=[X.values],  # 평가 데이터 (자기지도 학습이므로 동일 데이터 사용)
    max_epochs=200,
    patience=300,
    batch_size=512,
    warm_start=True
)

epoch 0  | loss: 0.3747  | val_0_unsup_loss_numpy: 1.0242999792099|  0:00:36s
epoch 1  | loss: 0.37369 | val_0_unsup_loss_numpy: 2.7688300609588623|  0:01:13s
epoch 2  | loss: 0.37567 | val_0_unsup_loss_numpy: 1.0334199666976929|  0:01:50s
epoch 3  | loss: 0.37592 | val_0_unsup_loss_numpy: 2.0231199264526367|  0:02:26s
epoch 4  | loss: 0.37606 | val_0_unsup_loss_numpy: 2.472249984741211|  0:03:02s
epoch 5  | loss: 0.37623 | val_0_unsup_loss_numpy: 1.8239699602127075|  0:03:38s
epoch 6  | loss: 0.3739  | val_0_unsup_loss_numpy: 0.6014900207519531|  0:04:14s
epoch 7  | loss: 0.37286 | val_0_unsup_loss_numpy: 1.7332600355148315|  0:04:51s
epoch 8  | loss: 0.37941 | val_0_unsup_loss_numpy: 0.8539800047874451|  0:05:26s
epoch 9  | loss: 0.37484 | val_0_unsup_loss_numpy: 4.489190101623535|  0:06:05s
epoch 10 | loss: 0.37186 | val_0_unsup_loss_numpy: 0.8140900135040283|  0:06:43s
epoch 11 | loss: 0.37438 | val_0_unsup_loss_numpy: 1.1226799488067627|  0:07:21s
epoch 12 | loss: 0.37407 | val_0_

KeyboardInterrupt: 

In [ ]:
reconstructed_data = pretrainer1.predict(X.values)[0]
mse_X = np.mean(np.power(X - reconstructed_data, 2), axis=1)
mae_X = np.mean(np.abs(X - reconstructed_data), axis=1)

data['mse_X'] = mse_X
data['mae_X'] = mae_X

n = 1000
df = data.sort_values(by='mae_X', ascending=False)
top_n_df = df.head(n)
accuracy = top_n_df[top_n_df['Class'] == 1].shape[0] / n
print('Top-{} 정확도: {}'.format(n, accuracy))

Top-1000 정확도: 0.295


In [ ]:
pretrainer1.save_model("/content/drive/MyDrive/tabnet_pretrainer")

Successfully saved model at /content/drive/MyDrive/tabnet_pretrainer1.zip


'/content/drive/MyDrive/tabnet_pretrainer1.zip'

In [ ]:
loaded_pretrainer1 = TabNetPretrainer()
loaded_pretrainer1.load_model("/content/drive/MyDrive/tabnet_pretrainer1.zip")

In [ ]:
reconstructed_data = loaded_pretrainer1.predict(X.values)[0]
mse_X = np.mean(np.power(X - reconstructed_data, 2), axis=1)
mae_X = np.mean(np.abs(X - reconstructed_data), axis=1)

data['mse_X'] = mse_X
data['mae_X'] = mae_X

n = 1000
df = data.sort_values(by='mae_X', ascending=False)
top_n_df = df.head(n)
accuracy = top_n_df[top_n_df['Class'] == 1].shape[0] / n
print('Top-{} 정확도: {}'.format(n, accuracy))

Top-1000 정확도: 0.295
